In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


## Адмитад

In [2]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:regionAreaName",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            #'region':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df = df.groupby(['week','utm_source','utm_campaign'],as_index=False).agg({'visits':'sum'})
df.to_excel(f'reports/трафик_адмитад.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:regionAreaName",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'region':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адмитад_регионы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_регионы.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:gender",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'gender':result[i]["dimensions"][3]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адмитад_пол.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_пол.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"


# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika


api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:interest",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"

    #filters = "ym:s:lastTrafficSource=='organic'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'interest':result[i]["dimensions"][3]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адмитад_иинтересы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_иинтересы.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"

# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:ageInterval",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
   # filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'age':result[i]["dimensions"][3]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адмитад_возраст.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_возраст.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"

# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:regionCountry",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'region':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df = df.groupby(['week','utm_source','utm_campaign','region'],as_index=False).agg({'visits':'sum'})
df.to_excel(f'reports/трафик_адмитад_страны.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_страны.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# ```По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
# Параметры запроса для библиотеки tapi_yandex_metrika```

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:bounce",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='admitad'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'bounce':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1

df.to_excel(f'reports/трафик_адмитад_отказы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_отказы.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:130: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:202: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:280: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-pack

## Адвакейк


In [3]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:regionAreaName",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'utm_content':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df = df.groupby(['week','utm_source','utm_campaign'],as_index=False).agg({'visits':'sum'})
df.to_excel(f'reports/трафик_адвакейк.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:regionAreaName",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'region':result[i]["dimensions"][4]["name"],
        'visits':result[i]["metrics"][0],
        'utm_content':result[i]["dimensions"][3]["name"]
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адвакейк_регионы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк_регионы.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:gender",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'gender':result[i]["dimensions"][4]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        'utm_content':result[i]["dimensions"][3]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адвакейк_пол.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(9) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк_пол.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"


# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika


api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:interest",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"

    #filters = "ym:s:lastTrafficSource=='organic'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'interest':result[i]["dimensions"][4]["name"],
            'utm_content':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0]
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адвакейк_иинтересы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк_иинтересы.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"

# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:ageInterval",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
   # filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
            'utm_campaign':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        'age':result[i]["dimensions"][4]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        'utm_content':result[i]["dimensions"][3]["name"]
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df.to_excel(f'reports/трафик_адвакейк_возраст.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адмитад_возраст.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"

# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:regionCountry",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'region':result[i]["dimensions"][4]["name"],
        'visits':result[i]["metrics"][0],
        'utm_content':result[i]["dimensions"][3]["name"]
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1


df = df.groupby(['week','utm_source','utm_campaign','region'],as_index=False).agg({'visits':'sum'})
df.to_excel(f'reports/трафик_адвакейк_страны.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк_страны.xlsx')
set_with_dataframe(worksheet, df1)

from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# ```По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
# Параметры запроса для библиотеки tapi_yandex_metrika```

api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:UTMCampaign,ym:s:UTMContent,ym:s:bounce",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource=='advcake'"
    #filters = "ym:pv:URL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'utm_source':result[i]["dimensions"][1]["name"],
        'utm_campaign':result[i]["dimensions"][2]["name"],
            'bounce':result[i]["dimensions"][4]["name"],
        'visits':result[i]["metrics"][0],
        'utm_content':result[i]["dimensions"][3]["name"]
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1

df.to_excel(f'reports/трафик_адвакейк_отказы.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1pfq7udxc20Hb1mlqZHFnE_jCTa0U0mYxoPHDw4Fvct0')
worksheet = sh.get_worksheet(13) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel(f'reports/трафик_адвакейк_отказы.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:130: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:202: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:280: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-pack

In [4]:
df

,date,utm_source,utm_campaign,bounce,visits,utm_content,week
0,2021-01-04,advcake,affiliate,Non-bounce,12.0,gorodtroika,2
1,2021-01-04,advcake,affiliate,Bounce,7.0,gorodtroika,2
2,2021-01-04,advcake,affiliate,Non-bounce,16.0,workle,2
3,2021-01-04,advcake,affiliate,Bounce,1.0,workle,2
4,2021-01-04,advcake,cityads,Non-bounce,4.0,26e7,2
...,...,...,...,...,...,...,...
10162,2021-12-13,advcake,saleads,Non-bounce,1.0,55b742e0-18af-11ec-9c01-a1b70db0b6d8,51
10163,2021-12-13,advcake,saleads,Non-bounce,3.0,626e9a10-fce0-11eb-a54b-1d9d5fe93b32,51
10164,2021-12-13,advcake,unicom24,Non-bounce,4.0,180390,51
10165,2021-12-13,advcake,unicom24,Bounce,17.0,180390,51
